In [2]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM, TimeDistributed, Reshape
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
num_classes = 10

# Normalize input images and convert labels to binary class matrices
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# Load pre-trained ResNet-50 without top (classification) layer
resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Freeze the layers in the ResNet-50 model
for layer in resnet.layers:
    layer.trainable = False

# Extract features from the images using ResNet-50
x_train_features = preprocess_input(x_train)
x_test_features = preprocess_input(x_test)

# Reshape the features for LSTM input
x_train_features = Reshape((1, 1, 2048))(resnet.predict(x_train_features))
x_test_features = Reshape((1, 1, 2048))(resnet.predict(x_test_features))

# Define RNN model architecture
input_shape = (1, 1, 2048)
rnn_input = Input(shape=input_shape)
rnn_layer = Reshape((1, 2048))(rnn_input)
rnn_layer = LSTM(64)(rnn_layer)
output_layer = Dense(num_classes, activation='sigmoid')(rnn_layer)
model = Model(inputs=rnn_input, outputs=output_layer)

# Compile and train the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train_features, y_train, batch_size=128, epochs=100, validation_data=(x_test_features, y_test))

# Evaluate the model
_, accuracy = model.evaluate(x_test_features, y_test)
print("Accuracy: %.2f%%" % (accuracy * 100))

313/313 [==============================] - 26s 84ms/step
Epoch 1/100
391/391 [==============================] - 7s 11ms/step - loss: 0.3279 - accuracy: 0.1438 - val_loss: 0.3148 - val_accuracy: 0.1922
Epoch 2/100
391/391 [==============================] - 6s 14ms/step - loss: 0.3069 - accuracy: 0.2308 - val_loss: 0.3003 - val_accuracy: 0.2583
Epoch 3/100
391/391 [==============================] - 6s 15ms/step - loss: 0.2967 - accuracy: 0.2640 - val_loss: 0.2925 - val_accuracy: 0.2901
Epoch 4/100
391/391 [==============================] - 5s 14ms/step - loss: 0.2908 - accuracy: 0.2857 - val_loss: 0.2882 - val_accuracy: 0.2995
Epoch 5/100
391/391 [==============================] - 5s 14ms/step - loss: 0.2865 - accuracy: 0.3005 - val_loss: 0.2845 - val_accuracy: 0.3078
Epoch 6/100
391/391 [==============================] - 5s 12ms/step - loss: 0.2833 - accuracy: 0.3101 - val_loss: 0.2828 - val_accuracy: 0.3087
Epoch 7/100
391/391 [==============================] - 5s 12ms/step - loss: 0.2